In [14]:
import pandas as pd
import re
import html
import nltk
#nltk.download() #only for the first time running it
from nltk.corpus import stopwords
from nltk import PorterStemmer
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json
import regex as re
import textblob as TextBlob

In [ ]:
#create dataframe on media posts
def createDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en' and len(row.get('referenced_tweets', [])) == 0:
                    yield (row['id'],
                            row['text'],
                            row['public_metrics']['retweet_count'],
                            row['public_metrics']['quote_count'],
                            row['public_metrics']['reply_count'],
                            row['public_metrics']['like_count'],
                            row['attachments']['media_keys'])
            except KeyError:
                pass

#create csv
df = pd.DataFrame(createDataFrame())
df.columns =['id', 'text', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'media_keys']
df.to_json('datain/nft_tweets.jsonl', orient='records', index=True, lines= True)
output = pd.read_json("datain/nft_tweets.jsonl", lines = True)
output['total'] = output[['retweet_count', 'quote_count','reply_count', 'like_count']].sum(axis=1)
#sort by highest total
output = output.sort_values(by = 'total', ascending = False)
#top 100
output = output.head(100)
output.to_json('datain/nft_top_100_tweets.jsonl', orient='records', index=True, lines= True)

In [3]:
#import the data
data = pd.read_json("datain/nft_top_100_tweets.jsonl", lines=True)
#data = pd.read_json("datain/nft_search_tweets_sample/nft_search_tweets_sample.jsonl", lines=True)
#data = data.iloc[0:100000]
data

,id,text,retweet_count,quote_count,reply_count,like_count,media_keys,total
0,1369983392155009024,Celebrating LilMoon Rockets #NFT Reveal Givewa...,917,186,1920,1076,[3_1369983389210710019],4099
1,1395797789662187520,Fleek is officially running on the #InternetCo...,820,22,57,2522,[3_1395783651007377411],3421
2,1393223147630075904,$SFT is now live and ready to trade on #Uniswa...,1545,5,5,865,[3_1393223052079546377],2420
3,1375754272437116928,Let’s knock NFTs on the head. I want...some re...,762,48,26,1326,[7_1375754125414166528],2162
4,1380255678925742080,New drop coming tomorrow :) Stay tuned!!! #NFT...,97,7,35,1836,[3_1380255253862379521],1975
...,...,...,...,...,...,...,...,...
95,1398432709287030784,💎 Genesis.json 💎\n\n✨ Artwork by @BT\n💰 Sold t...,27,6,8,126,[3_1398432708418805766],167
96,1394577401137569792,"You know I love you guys, so every week, start...",34,4,65,64,[16_1394577383311745028],167
97,1374012202508689408,NFTs are in the works for Funko. Will be inter...,9,3,54,100,[3_1374012197332918274],166
98,1376921740849209344,Woke up this morning to the WONDERFUL surprise...,20,13,20,112,"[3_1376921334437928960, 3_1376921406638682116]",165


In [4]:
#prep stop words
stop_words = stopwords.words('english')
stop_words.append('rt')
stop_words.append('nft')
# stop_words.append('#nft')

#function for cleaning a tweet (remove mentions, hashtags, links, html entities, stop words. And make sure it's only letters)
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        tweet = str.lower(tweet)
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)", " ", tweet).split()) # remove mentions and hashtags
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)", " ", tweet).split())
        tweet = re.sub("(http\S+|http)", "", tweet, flags=re.MULTILINE) # remove links
        tweet = re.sub('\&\w+', "", tweet) # remove html entities
        tweet = re.sub('[^a-zA-Z# ]+', ' ', tweet) # make sure tweet is only letters
        # stem & remove stop words
        # tweet = ' '.join([PorterStemmer().stem(word=word) for word in tweet.split() if word not in stop_words])
        tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
        return tweet

#clean data
for i in data.index:
    text = data["text"][i]
    cleaned_text = clean_tweet(text)
    cleaned_text = html.unescape(cleaned_text)
    data["text"][i] = cleaned_text

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
data.to_json('datain/nft_top_100_cleaned_interactions.jsonl', orient='records', index=True, lines= True)

0.1648320887445887

In [43]:
output = pd.read_json("datain/nft_top_100_cleaned_interactions.jsonl", lines = True)
output = pd.DataFrame(output)
for i in output.index:
    print(output.iloc[i]['text'])

celebrating lilmoon rockets reveal giveway worth lucky winners get worth follow retweet like tag friends comment wallet address
fleek officially running allowing dapps run web speed serve web development platform building open web already host sites supporting eth dapps projects possible
sft live ready trade mcap circ supply price active utility platform information contract address xf ea e cece eb ebbaf bfbcec ac f
let knock nfts head want revenge please watch video original painting sold actual original hands concise information comments
new drop coming tomorrow stay tuned
dollar woods official music video
dame lillard first exclusive going sale opensea right celebrate giving away dame series moment currently worth enter follow tag friends
limited celebrating defi portfolio new support coming scv pro access retweet follow fav comment address nrv holders preferred winners result
every maybe generation new advancement comes along expands new medium lucky enough witness rise omi
uniqly 

In [30]:
#nft_top_100_sentiment <- Has interactions and sentiment totals

positivity = ''
def getSentiment():
    output = pd.read_json("datain/nft_top_100_cleaned_interactions.jsonl", lines = True)
    output = pd.DataFrame(output)
    for i in output.index:
        row = TextBlob(output.iloc[i]['text'])
        if row.sentiment.polarity >= 0.7:
                positivity = 'mostly_positive'
        elif row.sentiment.polarity <= -0.7:
            positivity = 'mostly_negative'
        elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
            positivity = 'negative'
        elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
                positivity = 'positive'
        else:
            positivity = 'nuetral'
        yield i, row.sentiment.polarity, row.sentiment.subjectivity, positivity
    # with open("datain/cleaned.txt", encoding='utf8') as f:
    #     for line in f:
    #         row = TextBlob(line)
    #         if row.sentiment.polarity >= 0.7:
    #             positivity = 'mostly_positive'
    #         elif row.sentiment.polarity <= -0.7:
    #             positivity = 'mostly_negative'
    #         elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
    #             positivity = 'negative'
    #         elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
    #              positivity = 'positive'
    #         else:
    #             positivity = 'nuetral'
    #         yield line, row.sentiment.polarity, row.sentiment.subjectivity, positivity

df = pd.DataFrame(getSentiment())
df.columns =['text', 'polarity', 'subjectivity', 'positivity']
data['polarity'] = df['polarity']
data['subjectivity'] = df['subjectivity']
data['positivity'] = df['positivity']
df.to_json('datain/nft_top_100_cleaned_interactions_sentiment.jsonl', orient='records', index=True, lines= True)

KeyError: 'id'

In [ ]:
#Ascending polarity
df2 = data.sort_values(by = 'polarity', ascending = True)
df2.to_json('datain/nft_top_100_polarity_ascending.jsonl', orient='records', index=True, lines= True)
data

In [ ]:
#Descending polarity
df2 = data.sort_values(by = 'polarity', ascending = False)
df2.to_json('datain/nft_top_100_polarity_ascending.jsonl', orient='records', index=True, lines= True)
data

In [ ]:
# Word cloud of top 100 NFT's
